In [4]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
import urllib.request

from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse
import time
import operator

## MtgMintCards

In [5]:
urls = ['http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/the-great-henge',
       'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/oko-thief-of-crowns',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/arcane-signet-brawl-deck',
       'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/questing-beast',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/brazen-borrower',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/knights-charge-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/shimmer-dragon-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/steelbane-hydra-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/murderous-rider',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/once-upon-a-time']


In [39]:
infoMtgmintCard = []

In [40]:
def Extract_MtgMintCards(url):
    codigo_fonte = rq.get(url,headers={'content-type': 'text/html'})
    texto = codigo_fonte.text
    soup = BeautifulSoup(texto,'html.parser')
    
    titulo = soup.find('h1',class_='pd-card-name').text
    #print(titulo)

    sets = soup.find('span',{'itemprop':'model'}).text
    #print(sets)

    categoria = soup.find('span',{'itemprop':"category"}).text
    #print(categoria)

    preço = '$ ' + soup.find('span',{'itemprop':'price'}).text
    #print(preço)

    divInfo = soup.find('div',class_="col-sm-6",style="margin-top: 4px;")

    raridade = divInfo.find('span',{'itemprop':'model'}).text
    #print(raridade)

    descrição = divInfo.find('span',{'itemprop':'description'}).text
    #print(descrição)

    marca = divInfo.find('span',{'itemprop':"brand"}).text
    #print(marca)
    
    return infoMtgmintCard.append(dict(zip(['Titulo','Set','Categoria','Preço','Raridade','Marca','Descrição'],[titulo,sets,categoria,preço,raridade,marca,descrição])))


In [41]:
for url in urls:
    Extract_MtgMintCards(url)

In [42]:
infoMtgmintCard

[{'Titulo': '\nThe Great Henge ',
  'Set': 'Throne of Eldraine',
  'Categoria': 'English Regular',
  'Preço': '$ 13.99',
  'Raridade': 'Mythic',
  'Marca': ' Magic: The Gathering',
  'Descrição': 'This spell costs  less to cast, where X is the greatest power among creatures you control.\n: Add . You gain 2 life.\nWhenever a nontoken creature enters the battlefield under your control, put a +1/+1 counter on it and draw a card.'},
 {'Titulo': '\nOko, Thief of Crowns ',
  'Set': 'Throne of Eldraine',
  'Categoria': 'English Regular',
  'Preço': '$ 42.99',
  'Raridade': 'Mythic',
  'Marca': ' Magic: The Gathering',
  'Descrição': '+2: Create a Food token.\n+1: Target artifact or creature loses all abilities and becomes a green Elk creature with base power and toughness 3/3.\n-5: Exchange control of target artifact or creature you control and target creature an opponent controls with power 3 or less.'},
 {'Titulo': '\nArcane Signet (Brawl Deck) ',
  'Set': 'Throne of Eldraine',
  'Categoria

In [44]:
data = pd.DataFrame(infoMtgmintCard,columns=['Titulo','Set','Categoria','Preço','Raridade','Marca','Descrição'])
data

,Titulo,Set,Categoria,Preço,Raridade,Marca,Descrição
0,\nThe Great Henge,Throne of Eldraine,English Regular,$ 13.99,Mythic,Magic: The Gathering,"This spell costs less to cast, where X is the..."
1,"\nOko, Thief of Crowns",Throne of Eldraine,English Regular,$ 42.99,Mythic,Magic: The Gathering,+2: Create a Food token.\n+1: Target artifact ...
2,\nArcane Signet (Brawl Deck),Throne of Eldraine,English Regular,$ 29.99,Common,Magic: The Gathering,: Add one mana of any color in your commander'...
3,\nQuesting Beast,Throne of Eldraine,English Regular,$ 21.99,Mythic,Magic: The Gathering,"Vigilance, deathtouch, haste\nQuesting Beast c..."
4,\nBrazen Borrower,Throne of Eldraine,English Regular,$ 18.99,Mythic,Magic: The Gathering,Flash\nFlying\nBrazen Borrower can block only ...
5,\nKnights' Charge (Brawl Deck),Throne of Eldraine,English Regular,$ 18.99,Rare,Magic: The Gathering,"Whenever a Knight you control attacks, each op..."
6,\nShimmer Dragon (Brawl Deck),Throne of Eldraine,English Regular,$ 18.99,Rare,Magic: The Gathering,Flying\nAs long as you control four or more ar...
7,\nSteelbane Hydra (Brawl Deck),Throne of Eldraine,English Regular,$ 18.99,Rare,Magic: The Gathering,Steelbane Hydra enters the battlefield with X ...
8,\nMurderous Rider,Throne of Eldraine,English Regular,$ 15.99,Rare,Magic: The Gathering,"Lifelink\nWhen Murderous Rider dies, put it on..."
9,\nOnce Upon a Time,Throne of Eldraine,English Regular,$ 14.99,Rare,Magic: The Gathering,If this spell is the first spell you've cast t...


## Scryfall

In [125]:
urlsScryfall = ['https://scryfall.com/card/c17/2/balan-wandering-knight','https://scryfall.com/card/ddu/1/ezuri-renegade-leader',
               'https://scryfall.com/card/a25/3/akromas-vengeance','https://scryfall.com/card/txln/2/illusion',
                'https://scryfall.com/card/c17/1/alms-collector','https://scryfall.com/card/hou/158/abandoned-sarcophagus',
               'https://scryfall.com/card/ori/127/abbot-of-keral-keep','https://scryfall.com/card/rna/151/absorb',
               'https://scryfall.com/card/hou/158/abandoned-sarcophagus','https://scryfall.com/card/wwk/20/stoneforge-mystic']


In [126]:
infoScryfall = []

In [127]:
def extract_Scryfall(url):
    codigo_fonte = rq.get(url,headers={'content-type': 'text/html'})
    texto = codigo_fonte.text
    soup = BeautifulSoup(texto,'html.parser')

    divInfo = soup.find('div',class_="card-text")
    #(divInfo)

    titulo = divInfo.find('h1',class_='card-text-title').text
    #print(titulo)

    descrição = divInfo.find('div',class_='card-text-oracle').text
    #print(descrição)

    sets = soup.find('span',class_='prints-current-set-name').text
    #print(sets)


    raridadeCategoria = soup.find('span',class_='prints-current-set-details').text
    raridadeCategoria = re.findall("[a-zA-Z0-9]+", raridadeCategoria)
    #print(raridadeCategoria)

    raridade = raridadeCategoria[1:-1]
    categoria = raridadeCategoria[-1]
    
    return infoScryfall.append(dict(zip(['Titulo','Set','Categoria','Raridade','Descrição'],[titulo,sets,categoria,raridade,descrição])))



In [128]:
for url in urlsScryfall:
    extract_Scryfall(url)
infoScryfall

[{'Titulo': '\n          Balan, Wandering Knight\n            {2}{W}{W}\n',
  'Set': '\n        Commander 2017 (C17)\n      ',
  'Categoria': 'English',
  'Raridade': ['Rare'],
  'Descrição': '\nFirst strike\nBalan, Wandering Knight has double strike as long as two or more Equipment are attached to it.\n{1}{W}: Attach all Equipment you control to Balan.\n'},
 {'Titulo': '\n          Ezuri, Renegade Leader\n            {1}{G}{G}\n',
  'Set': '\n        Duel Decks: Elves vs. Inventors (DDU)\n      ',
  'Categoria': 'English',
  'Raridade': ['Mythic', 'Rare'],
  'Descrição': '\n{G}: Regenerate another target Elf.\n{2}{G}{G}{G}: Elf creatures you control get +3/+3 and gain trample until end of turn.\n'},
 {'Titulo': "\n          Akroma's Vengeance\n            {4}{W}{W}\n",
  'Set': '\n        Masters 25 (A25)\n      ',
  'Categoria': 'English',
  'Raridade': ['Rare'],
  'Descrição': '\nDestroy all artifacts, creatures, and enchantments.\nCycling {3} ({3}, Discard this card: Draw a card.)\

In [129]:
dataScryfall = pd.DataFrame(infoScryfall,columns=['Titulo','Set','Categoria','Raridade','Descrição'])
dataScryfall

,Titulo,Set,Categoria,Raridade,Descrição
0,"\n Balan, Wandering Knight\n ...",\n Commander 2017 (C17)\n,English,[Rare],"\nFirst strike\nBalan, Wandering Knight has do..."
1,"\n Ezuri, Renegade Leader\n ...",\n Duel Decks: Elves vs. Inventors (DDU...,English,"[Mythic, Rare]",\n{G}: Regenerate another target Elf.\n{2}{G}{...
2,\n Akroma's Vengeance\n {4...,\n Masters 25 (A25)\n,English,[Rare],"\nDestroy all artifacts, creatures, and enchan..."
3,\n Illusion\n,\n Ixalan Tokens (TXLN)\n,English,[Common],\nWhen this creature becomes the target of a s...
4,\n Alms Collector\n {3}{W}\n,\n Commander 2017 (C17)\n,English,[Rare],\nFlash\nIf an opponent would draw two or more...
5,\n Abandoned Sarcophagus\n ...,\n Hour of Devastation (HOU)\n,English,[Rare],\nYou may cast nonland cards with cycling from...
6,\n Abbot of Keral Keep\n {...,\n Magic Origins (ORI)\n,English,[Rare],\nProwess (Whenever you cast a noncreature spe...
7,\n Absorb\n {W}{U}{U}\n,\n Ravnica Allegiance (RNA)\n,English,[Rare],\nCounter target spell. You gain 3 life.\n
8,\n Abandoned Sarcophagus\n ...,\n Hour of Devastation (HOU)\n,English,[Rare],\nYou may cast nonland cards with cycling from...
9,\n Stoneforge Mystic\n {1}...,\n Worldwake (WWK)\n,English,[Rare],\nWhen Stoneforge Mystic enters the battlefiel...


## Abugames

In [213]:
urlsAbu = ['https://abugames.com/magic-the-gathering/singles/product-detail/Aggressive-Urge-%252D-FOIL/10th-Edition/English/',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Abundance/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Abundance-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Academy-Researchers/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Academy-Researchers-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Adarkar-Wastes/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Afflict/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Afflict-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Agonizing-Memories/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Flowstone-Salamander/Tempest/English/']

In [214]:
def CrawlerDinâmico(url):
    #C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe C:\Users\jcss3\Downloads\chromedriver.exe 
    driver = webdriver.Chrome("C:\\Users\\jcss3\\Downloads\\chromedriver.exe ")
    driver.get(url)
    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    return res


In [215]:
infoAbu = []

In [221]:
def extractor_Abu(url):
    
    texto = CrawlerDinâmico(url)
    soup = BeautifulSoup(texto,'html.parser')
    
    divInfo = soup.find('div',class_='row first')
    #print(divInfo)

    titulo = divInfo.find('h1').text
    #print(titulo)

    sets = divInfo.find('div',class_="col-md-2 col-sm-2 col-xs-4").text
    #print(sets)

    descrição = soup.find('div',{'id':"originalTextundefined"}).text
    descrição

    info = soup.find('div',class_="tableHere ng-star-inserted")
    info.text.split()

    raridade = info.text.split()[-1]
    raridade = raridade[0:]
    #print(raridade)

    tipo = info.text.split()[1:4]
    #tipo = tipo[:-1]
    #print(tipo)

    #regexMoney = re.compile(r'(\$)(\d+)(.)(\d{2})')
    return infoAbu.append(dict(zip(['Titulo','Set','Raridade','Descrição','Tipo'],[titulo,sets,raridade,descrição,tipo])))


In [222]:
for url in urlsAbu:
    extractor_Abu(url)


In [223]:
dataAbu = pd.DataFrame(infoAbu,columns=['Titulo','Set','Raridade','Descrição','Tipo'])
dataAbu

,Titulo,Set,Raridade,Descrição,Tipo
0,Aggressive Urge - FOIL,10th Edition,ommon,Target creature gets +1/+1 until end of turn.\...,FOIL10thEditio
1,Aggressive Urge - FOIL,10th Edition,ommon,Target creature gets +1/+1 until end of turn.\...,"[FOIL, 10th, Edition, Instant]"
2,Abundance,10th Edition,are,"If you would draw a card, you may instead choo...",[Enchantment]
3,Abundance - FOIL,10th Edition,are,"If you would draw a card, you may instead choo...","[10th, Edition, Enchantment]"
4,Academy Researchers,10th Edition,Uncommon,"When Academy Researchers comes into play, you ...","[Edition, Creature, -, Human, Wizard2, /]"
5,Academy Researchers - FOIL,10th Edition,Uncommon,"When Academy Researchers comes into play, you ...","[FOIL, 10th, Edition, Creature, -, Human, Wiza..."
6,Adarkar Wastes,10th Edition,are,: Add to your mana pool.\n: Add or to your ...,"[Edition, Land]"
7,Afflict,10th Edition,ommon,Target creature gets -1/-1 until end of turn.\...,[Instant]
8,Afflict - FOIL,10th Edition,ommon,Target creature gets -1/-1 until end of turn.\...,"[10th, Edition, Instant]"
9,Agonizing Memories,10th Edition,ncommon,Look at target player's hand and choose two ca...,"[Edition, Sorcery]"
